In [ ]:
import asyncio
import logging
import sys

from astropy.coordinates import ICRS, Angle
from astropy.time import Time
import astropy.units as u

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.comcam import ComCam

import mt_utils

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

comcam = ComCam(domain)
mtptg = salobj.Remote(domain, "MTPtg")
newmtmount = salobj.Remote(domain, "NewMTMount")

await asyncio.gather(comcam.start_task, mtptg.start_task, newmtmount.start_task)

In [ ]:
target_name = "HD 185975"
ra = "20 28 18.7402"
dec = "-87 28 19.938"

radec_icrs = ICRS(Angle(ra, unit=u.hourangle), Angle(dec, unit=u.deg))

In [ ]:
mtptg.cmd_raDecTarget.set(
    ra=radec_icrs.ra.hour,
    declination=radec_icrs.dec.deg,
    rotPA=Angle(0.0, unit=u.deg).deg,
    targetName=target_name,
    frame=2,
    epoch=2000,
    equinox=2000,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotFrame=1,
    rotMode=1,
)

In [ ]:
await mt_utils.slew(mtptg, newmtmount, True)
#ack = await mtptg.cmd_raDecTarget.start(timeout=30.)
#print(ack)

In [ ]:
await comcam.take_object(exptime=30.0, n=2)